# 决策树

一般流程  
1. 收集数据：可以使用任何方法。  
2. 准备数据：树构造算法只适用于标称型数据，因此数值型数据必须离散化。  
3. 分析数据：可使用任何方法，构造树完成之后，我们应该检查图形是否符合预期。  
4. 训练算法：构造树的数据结构。  
5. 测试算法：使用经验树计算错误率。  
6. 使用算法：次步骤可以适用于任何监督学习算法，而使用决策树可以更好地理解数据的内在含义。

本书使用ID3算法划分数据集（一些决策树算法采用二分法划分数据）

### 创建简单的鱼鉴定数据集

In [10]:
data_set = [
    [1,1,'yes'],
    [1,1,'yes'],
    [1,0,'no'],
    [0,1,'no'],
    [0,1,'no']
]
labels = ['能够不浮出水面', '有鳍']

In [12]:
from decision_trees_practice.trees import calc_shannon_ent

calc_shannon_ent(data_set)

0.9709505944546686

熵：度量数据集的无序程度  
熵越高，则混合的数据越多，在数据集中添加更多的分类，熵会如何变化

In [3]:
data_set1 = data_set
data_set1.append([1,1,'maybe'])

calc_shannon_ent(data_set1)

1.4591479170272448

In [6]:
def split_data_set(data_set, axis, value):
    """
    按照给定特征划分数据集，筛选某个特征为指定特征值的数据
    （然后因为是按该特征进行划分了，
    该特征在以后的划分中就不用再出现，所以把该特征在新的列表中移除）
    :param data_set: 
    :param axis: 
    :param value: 
    :return: 
    """
    ret_data_set = []
    for feature_vec in data_set:
        if feature_vec[axis] == value:
            # 抽取, 把指定特征从列表中去掉，组成一个新的特征+标签的列表
            reduced_feature_vec = feature_vec[:axis]
            reduced_feature_vec.extend(feature_vec[axis+1:])
            ret_data_set.append(reduced_feature_vec)
    return ret_data_set

In [7]:
myDat = [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
split_data_set(myDat,0,1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [8]:
split_data_set(myDat,0,0)

[[1, 'no'], [1, 'no']]

## 选择最好的数据集划分方式  
遍历整个数据集，循环计算香农熵和split_data_set函数，找到最好的特征划分方式。熵计算将会告诉我们如何划分数据集是最好的数据组织方式  

使用"信息增益最大/熵最小"的特征进行划分

In [14]:
def choose_best_feature_to_split(data_set):
    # 特征数，取第一个样本即可得知，由于最后一个元素是标签，所以特征数是长度-1
    num_features = len(data_set[0]) - 1
    
    # 计算整个数据集的熵（无序度）
    base_entropy = calc_shannon_ent(data_set)
    
    best_info_gain = 0.0
    best_feature = -1
    
    # 遍历数据集的特征1，组成一个新的数组1， 遍历数据集的特征2，组成一个新的数组2...
    # 我的理解是，收集每一个特征都会有哪些特征值
    for i in range(num_features):
        # 创建唯一的分类标签列表
        feature_list = [example[i] for example in data_set]
        
        # 每一组特征值列表中，去掉重复的特征值
        unique_vals = set(feature_list)
        new_entropy = 0.0
        
        # 计算每种划分方式的信息熵
        for value in unique_vals:
            # 原数据集剔除了某个特征值之后的数据集
            sub_data_set = split_data_set(data_set, i, value)
            
            # 该特征值在数据集中出现的概率
            prob = len(sub_data_set)/float(len(data_set))
            
            # 计算划分后的子数据集的熵值（信息期望值总和）
            new_entropy += prob * calc_shannon_ent(sub_data_set)
            
        # 整个数据集的熵，减去划分后的子数据集的熵，得出的是信息增益？这是什么东西呢？
        # 为什么是减？-- 信息增益是熵的减少或者是数据无序度的减少
        info_gain = base_entropy - new_entropy
        
        if (info_gain > best_info_gain):
            # 计算最好的信息增益
            best_info_gain = info_gain
            best_feature = i
    return best_feature

In [15]:
choose_best_feature_to_split(myDat)

0

就是说按第一个特征进行划分效果最好

## 递归构建决策树